In [76]:
import pandas as pd 
import numpy as np 
import plotly.express as px
from pathlib import Path

In [77]:
# Subset of patients to explore, 3 prediabetic and 3 not, 3 female and 3 male
patients = [1, 2, 4, 9, 14]

In [78]:
# Load in demographic data
demo = pd.read_csv(Path('data') / 'Demographics.csv')
key = demo[demo["ID"].isin([1,2,4,9,14])]
key

,ID,Gender,HbA1c
1,1,FEMALE,5.5
3,4,FEMALE,6.4
5,2,MALE,5.6
10,9,MALE,6.1
13,14,MALE,5.5


In [97]:
key[key["ID"] == 4]["HbA1c"].iloc[0]

np.float64(6.4)

In [79]:
# Each element of the list is glucose data for a specfic patient
dex = []
for i, v in enumerate(patients):    
    dex.append(pd.read_csv(Path('data') / 'dex' / f'Dexcom_{str(v).zfill(3)}.csv'))

# Clean the data 
for i, v in enumerate(dex):
    dex[i] = dex[i].dropna(subset=["Timestamp (YYYY-MM-DDThh:mm:ss)"])
    dex[i] = dex[i][["Timestamp (YYYY-MM-DDThh:mm:ss)", "Glucose Value (mg/dL)"]]
    dex[i]["Timestamp (YYYY-MM-DDThh:mm:ss)"] = pd.to_datetime(dex[i]["Timestamp (YYYY-MM-DDThh:mm:ss)"])

In [86]:
food = []
for i, v in enumerate(patients):    
    food.append(pd.read_csv(Path('data') / 'food' / f'Food_Log_{str(v).zfill(3)}.csv'))

for i, v in enumerate(food):
    food[i]["time_begin"] = pd.to_datetime(food[i]["time_begin"])

In [93]:
hr = []
for i, v in enumerate(patients):    
    hr.append(pd.read_csv(Path('data') / 'hr' / f'HR_{str(v).zfill(3)}.csv'))

for i, v in enumerate(hr):
    hr[i]["datetime"] = pd.to_datetime(hr[i]["datetime"])

C:\Users\1khal\AppData\Local\Temp\ipykernel_15160\1775773120.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  hr[i]["datetime"] = pd.to_datetime(hr[i]["datetime"])


Create a line graph showing heart rate overtime, for simplicity we will only consider the first day (different for each patient). Plot the heart rates of each patient, coloring in based on prediabetes. 

In [83]:
a = pd.to_datetime(hr[0]["datetime"], format="%m/%d/%y %H:%M")
first_day = a.iloc[0].day
first_day

13

In [84]:
a

0        2020-02-13 15:29:00
1        2020-02-13 15:29:00
2        2020-02-13 15:29:00
3        2020-02-13 15:29:00
4        2020-02-13 15:29:00
                 ...        
634183   2020-02-22 17:55:00
634184   2020-02-22 17:56:00
634185   2020-02-22 17:56:00
634186   2020-02-22 17:56:00
634187   2020-02-22 17:56:00
Name: datetime, Length: 634188, dtype: datetime64[ns]

In [108]:
# Create a new df
hr_df_1 = pd.DataFrame()
# For each patient:
for i, v in enumerate(patients):
#   Query down heart rates for only the first day
    first_day = hr[i]["datetime"].iloc[0].day
    temp = hr[i][hr[i]["datetime"].dt.day == first_day]
#   Add column for prediabeticness
    prediabetic = False
    if key[key["ID"] == v]["HbA1c"].iloc[0] > 5.7:
        prediabetic = True

    prediabetic_col = [prediabetic] * temp.shape[0]
    id_col = [v] * temp.shape[0]
    temp["prediabetic"] = pd.Series(prediabetic_col)
    temp["ID"] = pd.Series(id_col)
#   Concatenate with df
    hr_df_1 = pd.concat([hr_df_1, temp])
# Sort and remove duplicates



C:\Users\1khal\AppData\Local\Temp\ipykernel_15160\1617653222.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\1khal\AppData\Local\Temp\ipykernel_15160\1617653222.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\1khal\AppData\Local\Temp\ipykernel_15160\1617653222.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [109]:
# Plot
fig = px.line(
    hr_df_1,
    x='datetime',
    y=' hr',
    color='prediabetic',      # color based on diabetes status
    line_group='ID',  # separate line per patient
    title='Heart Rate Over Time by Patient and Prediabetes Status',
    labels={'prediabetic': 'Prediabetic Status'}
)

fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed